<a href="https://colab.research.google.com/github/KimuYounguWoo/machine-learning-programming/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5-2 교차 검증과 그리드 서치
## 목표
1. 검증 세트가 필요한 이유에 대해서 공부
2. 교차 검증
3. 그리드 서치와 랜덤 서치

## DT의 하이퍼 파라미터
DT의 하이퍼 파라미터를 검증 세트와 그리드 서치를 이용해 찾아내기

### 데이터 준비

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

훈련 세트는 60, 검증 세트는 20, 테스트 테스틑 20으로 분할해준다.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


학습에 과적합된 과대적합 모델 생성

### 교차 검증
훈련 세트는 많으면 많을 수록 좋기에, 교차 검증을 통해 안정적인 점수를 얻어봅시다.

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01221514, 0.01106215, 0.0122602 , 0.01139927, 0.01759267]), 'score_time': array([0.00195169, 0.00190711, 0.00199699, 0.0018363 , 0.0018754 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


점수가 좋아진 것을 확인할 수 있습니다.

여기서 교차 검증에 쓰이는 알고리즘은 k-겹 교차 검증으로
1. 훈련 세트에서 검증 세트를 떼어내는 과정에서 검증 세트를 여러 경우의 수로 분리 ( V, T, T ), ( T, V, T ), ( T, T, V )
2. 이 경우의 수로 나온 점수의 평균을 최종 점수로 사용
과 같은 방식으로 사용합니다.

여기서 주의할 점은 훈련 세트를 섞어 폴드를 나누지 않습니다.<BR>우리는 함수를 이용해 이미 섞은 후에 준비했지만, 만약 데이터세트가 섞어야 한다면 아래와 같이 분할기를 사용해줘야 합니다.

In [ ]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


n_splits은 K-겹 검증에서 K를 설정해줍니다

### 하이퍼 파라미터 찾기
지금까지는 하이퍼 파라미터를 찾을 때, 반복문을 사용해서 찾았씁니다<BR>
하지만 이런 방식에는 Cost가 너무 높게 나와 사이킷런에서는 그리드 서치를 제공합니다<br>

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}


이처럼 파라미터를 설정해놓습니다

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

그리드 서치 객체를 만듭니다. 여기서는 없지만 cv는 파라미터 당 cv만큼 교차 검증을 실시합니다<br>
그래서 위 파라미터로 실행할 경우 params(5) * k(5)로 총 25번 훈련합니다.

In [ ]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

이제 베스트를 찾아야합니다. 이것또한 gs에서 지원해줍니다

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


이 조건에서의 훈련은 96%가 가장 높은 점수이고

In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


그 점수를 주는 params는 0.0001인 것을 알 수 있습니다.

In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


당연히, 다른 파라미터의 점수도 알 수 있습니다.

In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


이런식으로 함수를 만들어서 사용한다면, 가장 높은 점수의 파라미터를 알 수 있습니다.

이번에는 좀 더 많은 params를 지정해보겠습니다.

In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }


깊이와 분할하기 위한 최소 샘플의 수, 불순도를 놓고 진행합니다<br>
이번에는 10 * 5 * 15 * 10으로 굉장히 많은 훈련을 진행합니다

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


이렇게 많은 반복을 진행할 때는 시간이 많이 걸리니, 파라미터의 범위를 잘 지정해줘야 합니다.

### 랜덤 서치
랜덤 서치는 그리드 서치에서 값의 범위나 간격을 정하기 힘들거나, 너무 많은 조건이 있어 수행 시간이 오래 걸릴 때 사용합니다

In [ ]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([6, 7, 5, 3, 6, 5, 3, 7, 1, 4])

두 모듈 모두, 무작위 값을 뽑아줍니다.

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.25451372, 0.33639972, 0.07744392, 0.59258797, 0.82752534,
       0.73562562, 0.00202098, 0.40331049, 0.87585306, 0.62324766])

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79cef5d6ac50>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x79cef5d690c0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79cef5d69000>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79cef677a7a0>},
                   random_state=42)

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [ ]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


비슷한 점수가 나왔네요, 앞으로는 그리드 서치를 활용해 매개변수를 지정해봅시다.